#### Installing dependencies

In [1]:
!pip install numpy torch matplotlib

#### Importing packages

In [2]:
%matplotlib inline
import numpy as np
import torch
import torch.utils.data as Data
from torch.autograd import Variable
import matplotlib.pyplot as plt
from collections import OrderedDict
from math import pi

#### NN class for Regression

In [3]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize, nhiddenLayers, hiddenNeurons):
        super(linearRegression, self).__init__()
        layers = []

        for i in range(0,nhiddenLayers): # adding hidden layers
            if(i==0): #Input layer
                layers.append(torch.nn.Linear(inputSize, hiddenNeurons))
                layers.append(torch.nn.ReLU())
            else:
                layers.append(torch.nn.Linear(hiddenNeurons, hiddenNeurons))
                layers.append(torch.nn.ReLU())

        layers.append(torch.nn.Linear(hiddenNeurons, outputSize)) # output layer
        self.model = torch.nn.Sequential(*layers)
        print(self.model)

    def forward(self, x):
        out = self.model(x)
        return out

#### NN Parameters

In [4]:
learningRate = 0.001
momentum = 0.9 
epochs = 2000
batch_size = 3000
train_set = 60000
test_set = 60000

#### Making the Dataset

In [5]:
f1 = lambda x: 2 * (x ** 2) - 1
func = lambda x: f1(f1(np.cos(x)))
x = torch.unsqueeze(torch.linspace(-2 * pi, 2 * pi, train_set), dim=1)
y = func(x)

# Train Dataset, ,Making generator for batches
train_dataset = Data.TensorDataset(x, y)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Test Dataset
test_ind = np.random.choice(train_set, test_set)
test_x = x[test_ind]
test_y = func(test_x)

#### Defining Net, Criterion and Optimizer

In [6]:
net = linearRegression(1,1,3, 24)
criterion = torch.nn.MSELoss() # Define criterion to evaluate the network
optimizer = torch.optim.SGD(net.parameters(), lr=learningRate, momentum=momentum)

Sequential(
  (0): Linear(in_features=1, out_features=24, bias=True)
  (1): ReLU()
  (2): Linear(in_features=24, out_features=24, bias=True)
  (3): ReLU()
  (4): Linear(in_features=24, out_features=24, bias=True)
  (5): ReLU()
  (6): Linear(in_features=24, out_features=1, bias=True)
)


#### Training the NN

In [7]:
for epoch in range(epochs): # Iteration in each epoch
    for step, (t_x, t_y) in enumerate(train_loader): # Iterating in each batch
        optimizer.zero_grad() # Removing buffer from previous epochs
        outputs = net(t_x) # Output training into the model
        loss = criterion(outputs, t_y) # Get loss for predicted outputs
        print("Epoch: ", epoch,"  Step: ", step,"  Loss: ", loss)
        loss.backward() # Propagate the loss
        optimizer.step() # Update parameters

Epoch:  0   Step:  0   Loss:  tensor(0.5181, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  1   Loss:  tensor(0.5168, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  2   Loss:  tensor(0.5116, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  3   Loss:  tensor(0.5117, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  4   Loss:  tensor(0.5125, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  5   Loss:  tensor(0.5083, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  6   Loss:  tensor(0.4883, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  7   Loss:  tensor(0.5130, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  8   Loss:  tensor(0.5085, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  9   Loss:  tensor(0.5091, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  10   Loss:  tensor(0.5035, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  11   Loss:  tensor(0.5069, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  12   Loss:  tensor(0.5083, grad_fn=<MseLossBackward>)
Epoch:  0   Step:  13   Loss:  tensor(0.4908, grad_fn=<MseLos

KeyboardInterrupt: 

#### Evaluating the model

In [ ]:
net = net.eval()
test_outputs = net(test_x)
mean_test_error = torch.mean(torch.abs(test_y-test_outputs))
print("Mean Test Error: ", mean_test_error)